# WebComponent Reference

In [ ]:
import param
import panel as pn
pn.extension()
pn.config.sizing_mode="stretch_width"
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Introduction

Web Components are powering the moderne web. For an introduction and overview see [Web Components: The Secret Ingredient Helping Power The Web](https://www.youtube.com/watch?v=YBwgkr_Sbx0).

You can find web components to use in your Panel Project at 

- [webcomponent.org](https://www.webcomponents.org/)
- [npm](https://www.npmjs.com/), 

Please note picking and using the right web component library and the right version can be **challenging** as the web component standard is still evolving. You might have to do some experimentation before it works for you.


## Google Model Viewer

Google has developed a `model-viewer` web component for interactively viewing very large and detailed 3D models. For an introduction see [modelviewer.dev](https://modelviewer.dev/)

We start out by defining the *javascript* nescessary to load the package and the web component *html* string.

##### TODO: MAYBE REMOVE SOME OF THE JS TO SIMPLIFY?

In [ ]:
js = """
<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js"></script>
<script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.js"></script>
<script nomodule src="https://unpkg.com/@google/model-viewer/dist/model-viewer-legacy.js"></script>
"""
js_pane = pn.pane.HTML(js)

html="""
<model-viewer src="https://modelviewer.dev/shared-assets/models/Astronaut.glb" alt="A 3D model of an astronaut"
auto-rotate camera-controls>
</model-viewer>
"""

### Simple Version

In [ ]:
model_viewer_pane = pn.pane.WebComponent(html=html, height=200, width=300)

pn.Column(
    js_pane,
    model_viewer_pane,
    height=200,
)

### Advanced Version

In [ ]:
astronaut_src = "https://modelviewer.dev/shared-assets/models/Astronaut.glb"
flight_helmet_src="https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/FlightHelmet/glTF/FlightHelmet.gltf"

class ModelViewer(pn.pane.WebComponent):
    """A Wired ModelViewer"""
    html = param.String(html)
    attributes_to_watch= param.Dict({"src": "src", "exposure": "exposure"})

    src = param.String(astronaut_src)
    exposure = param.Number(1.0, bounds=(0, 2))


model_viewer = ModelViewer(min_height=200, width=300, src=flight_helmet_src)

pn.Row(
    js_pane,
    model_viewer, pn.Param(model_viewer, parameters=["exposure"]),
)

For an even more interactive version that has been developed by Google see [model-viewer tester](https://modelviewer.dev/examples/tester.html).

## Open Source Web Component Libraries

- [Material](https://github.com/material-components/material-components-web-components#readme) (Work in Progress)
- [Vaadin](https://vaadin.com/components) (Mature)

## Including Prebuilt Web Component Modules using unpkg.com

If you find a web component library on [npm](https://npmjs.com) you can find a corresponding precombiled library on [unpkg](https://unpkg.com)

For example if you find the `wired-button` at [https://www.npmjs.com/package/wired-button](https://www.npmjs.com/package/wired-button) then you can browse the precompiled files at [https://www.unpkg.com/browse/wired-button/](https://www.unpkg.com/browse/wired-button/) to locate the relevant precombiled file at [https://www.unpkg.com/wired-button@2.0.0/lib/wired-button.js](https://www.unpkg.com/wired-button@2.0.0/lib/wired-button.js).

## References

- [Build an app with WebComponents in 9 minutes](https://www.youtube.com/watch?v=mTNdTcwK3MM)
- [How to use Web Components in a JavaScript project](https://www.youtube.com/watch?v=88Sa-SlHRxk&t=63s)